# Storing Chat log History 

In [12]:
import os
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
from datetime import datetime, timezone

## Load environmental variables

In [ ]:
if not load_dotenv(".env"):
    print("An error has occured. Make sure the file exists and is readable")
else:
    print("Loading successful")

Loading successful


## Establish connection to MongoDB server

##### Load database connection detail

In [14]:
def mongodb_connection():
    try:
        dbname = os.getenv("MONGO_DB_NAME")
        password = os.getenv("MONGO_DB_PASSWORD")
        uri = os.getenv("MONGO_DB_URI")

        if not all([dbname, password, uri]):
            print("An error has occured. MongoDB connection details not found in environmental variables")
            return None, None
        
        uri = uri.replace("<db_password>", password)

        # Send out a ping to verify connection status with MongoDB
        print("Establishing connection to MongoDB")
        client = MongoClient(uri, server_api=ServerApi('1'))
        client.admin.command('ping')
        print("Pinged your deployment. You have successfully connected to MongoDB")

        db = client[dbname]
        chat_sessions_collection = db['chat_sessions']
        print(f"Accessed database '{dbname}' and collection '{chat_sessions_collection}'.")

    except Exception as e:
        print(f"An error has occured when connecting to MongoDB or accessing collections: {e}")
        if client:
            try:
                client.close()

            except Exception as e:
                print(f"An error has occured when trying to close MongoDB after connection failure: {e}")
                
        return None, None
    
    return client, chat_sessions_collection                   

## Logging messages 

In [15]:
def log_messages(mongo_collection, session_ID, role, content, metadata = None):
    if mongo_collection is None:
        print("Cannot log message. MongoDB collection object is not valid\n")
        return False
    
    message_doc = {
        "role": role,
        "content": content,
        "timestamp":datetime.now(timezone.utc),
        "metadata": metadata or {}
    }

    try:
        mongo_collection.update_one(
            {"_id": session_ID},
            {
                "$push": {"messages": message_doc},
                "$setOnInsert": {
                    "_id": session_ID,
                    "start_time": datetime.now(timezone.utc)
                }
            },

            upsert = True
        )
        return True 

    except Exception as e:
        print(f"An error has occured logging MongoDB for session {session_ID}: {e}")
        return False

## Main Execution Block

In [16]:
if __name__ == "__main__":
    print("Initializing Chatlog History Process")

    # Declare globally
    client, chat_sessions_collection = mongodb_connection()

    # ONLY proceed if MongoDB's client and chat session collection access were successful
    if client is not None and chat_sessions_collection is not None:
        try:
            # Generate example
            session_id_1 = "session_abc_123"
            session_id_2 = "session_xyz_789"

            print(f"\nLogging messages for session: {session_id_1}")
            success1 = log_messages(chat_sessions_collection, session_id_1, "user", "Hello. How does photosynthesis work?")
            if success1: 
                print("Logged user message successfully.")

            print(f"\nLogging messages for session: {session_id_2}")
            success2 = log_messages(chat_sessions_collection, session_id_1, "assistant", "Greetings. Photosynthesis is the process where plants use sunlight, water...", {"model": "gpt-4"})
            if success2: print("Logged assistant message successfully.")

        except Exception as e:
            print("An error has occured during the main execution: {e}")
            print(f"Error Type: {type(e).__name__}")
        
        finally:
            if client:
                try:
                    print("\nClosing MongoDB connection.")
                    client.close()
                    print("\nMongoDB connection has been closed succesfully")

                except Exception as e:
                    print("An error has occured when attempting to close MongoDB connection: {e}")

    else:
        print("An error has occured. Failure to connect to MongoDB or access chat collections")

    print("Chatlog History Process completed")


Initializing Chatlog History Process
Establishing connection to MongoDB
Pinged your deployment. You have successfully connected to MongoDB
Accessed database 'ielts-chatlog-history' and collection 'Collection(Database(MongoClient(host=['ac-jxwni1i-shard-00-00.t6b6p10.mongodb.net:27017', 'ac-jxwni1i-shard-00-02.t6b6p10.mongodb.net:27017', 'ac-jxwni1i-shard-00-01.t6b6p10.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='ielts-chatlog-history', authsource='admin', replicaset='atlas-telby1-shard-0', tls=True, server_api=<pymongo.server_api.ServerApi object at 0x000002ACD5182A20>), 'ielts-chatlog-history'), 'chat_sessions')'.

Logging messages for session: session_abc_123
Logged user message successfully.

Logging messages for session: session_xyz_789
Logged assistant message successfully.

Closing MongoDB connection.

MongoDB connection has been closed succesfully
Chatlog History Process completed
